#Libraries

In [1]:
import pandas as pd
import numpy as np 

In [2]:
df_train=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Competitions/Loan Prediction/train_ctrUa4K.csv')

In [3]:
df_train.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

# Data Cleaning

In [4]:
def categorical_to_binary(data,ohe_pa,ohe_dep):
  data['Gender']=data['Gender'].map({'Male':1,'Female':0})
  data['Married']=data['Married'].map({'Yes':1,'No':0})
  data['Education']=data['Education'].map({'Graduate':1,'Not Graduate':0})
  data['Self_Employed']=data['Self_Employed'].map({'Yes':1,'No':0})
  
  dependents=pd.DataFrame(ohe_dep.transform(data[['Dependents']]),columns=['0','1','2'])

  property_area=pd.DataFrame(ohe_pa.transform(data[['Property_Area']]),columns=['Urban','Rural'])
  data.drop(['Property_Area','Dependents'],axis=1,inplace=True)
  data=pd.concat([data,property_area,dependents],axis=1)

  return data 

In [5]:
def remove_missing_value_num(data,knn_num):
  missing_num=['LoanAmount','Loan_Amount_Term','Credit_History']
  missing_value1=pd.DataFrame(data=knn_num.transform(data[missing_num]),columns=missing_num)
  
  data.drop('CoapplicantIncome',axis=1,inplace=True)
  data.drop(missing_num,axis=1,inplace=True)
  data=pd.concat([data,missing_value1],axis=1)

  return data  

In [6]:
def remove_missing_value_cat(data,si):
  missing_col=['Gender','Married','Dependents','Self_Employed']
  filled_values=pd.DataFrame(data=si.transform(data[missing_col]),columns=missing_col)

  data.drop(missing_col,axis=1,inplace=True)
  data=pd.concat([data,filled_values],axis=1)

  return data

In [7]:
def upsampling(data):
  data_majority=data[data['Loan_Status']=='Y']
  data_minority=data[data['Loan_Status']=='N']

  from sklearn.utils import resample

  data_minority_upsampled = resample(data_minority, 
                                 replace=True,   
                                 n_samples=len(data_majority),    
                                 random_state=123)
  
  data=pd.concat([data_majority,data_minority_upsampled])

  return data

# Implementing the functions

In [8]:
missing_col=['Gender','Married','Dependents','Self_Employed']
from sklearn.impute import SimpleImputer
si=SimpleImputer(strategy='most_frequent')
si.fit(df_train[missing_col])

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='most_frequent', verbose=0)

In [9]:
df_train=remove_missing_value_cat(df_train,si)

In [10]:
from sklearn.preprocessing import OneHotEncoder
ohe_pa=OneHotEncoder(sparse=False,drop='first')
ohe_pa.fit(df_train[['Property_Area']])

OneHotEncoder(categories='auto', drop='first', dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=False)

In [11]:
ohe_dep=OneHotEncoder(sparse=False,drop='first')
ohe_dep.fit(df_train[['Dependents']])

OneHotEncoder(categories='auto', drop='first', dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=False)

In [12]:
from sklearn.impute import KNNImputer
knn_impute_num=KNNImputer()
missing_num=['LoanAmount','Loan_Amount_Term','Credit_History']
knn_impute_num.fit(df_train[missing_num])

KNNImputer(add_indicator=False, copy=True, metric='nan_euclidean',
           missing_values=nan, n_neighbors=5, weights='uniform')

In [13]:
df_train=categorical_to_binary(df_train,ohe_pa,ohe_dep)

In [14]:
df_train=remove_missing_value_num(df_train,knn_impute_num)

In [15]:
#df_train=upsampling(df_train)

# Model creation

In [16]:
df_train.head()

,Loan_ID,Education,ApplicantIncome,Loan_Status,Gender,Married,Self_Employed,Urban,Rural,0,1,2,LoanAmount,Loan_Amount_Term,Credit_History
0,LP001002,1,5849,Y,1,0,0,0.0,1.0,0.0,0.0,0.0,164.6,360.0,1.0
1,LP001003,1,4583,N,1,1,0,0.0,0.0,1.0,0.0,0.0,128.0,360.0,1.0
2,LP001005,1,3000,Y,1,1,1,0.0,1.0,0.0,0.0,0.0,66.0,360.0,1.0
3,LP001006,0,2583,Y,1,1,0,0.0,1.0,0.0,0.0,0.0,120.0,360.0,1.0
4,LP001008,1,6000,Y,1,0,0,0.0,1.0,0.0,0.0,0.0,141.0,360.0,1.0


In [17]:
df_train['Loan_Status']=df_train['Loan_Status'].map({'Y':1,'N':0})
y=df_train['Loan_Status']
x=df_train.drop(['Loan_Status','Loan_ID'],axis=1)

In [18]:
print(len(x))
print(len(y))

614
614


In [19]:
x.shape

(614, 13)

#Scaler

In [20]:
from sklearn.preprocessing import StandardScaler
mms=StandardScaler()
x_scaled=mms.fit_transform(x)

#Feature Selection

In [21]:
from sklearn.feature_selection import mutual_info_classif
relation_targets=pd.DataFrame(mutual_info_classif(x_scaled,y),columns=['Relation_with_target'])
relation_targets['columns']=x.columns

In [33]:
relation_targets.sort_values('Relation_with_target',ascending=False)

,Relation_with_target,columns
12,0.142884,Credit_History
4,0.034897,Self_Employed
8,0.027824,1
7,0.026380,0
9,0.026108,2
0,0.017064,Education
11,0.016081,Loan_Amount_Term
1,0.009216,ApplicantIncome
5,0.000673,Urban
2,0.000000,Gender


In [23]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
skb=SelectKBest(score_func=chi2,k=x.shape[1])

In [30]:
from sklearn.preprocessing import MinMaxScaler
mm_scaler=MinMaxScaler()
x_mm=mm_scaler.fit_transform(x)
skb.fit(x_mm,y)

SelectKBest(k=13, score_func=<function chi2 at 0x7f3bed1ac7b8>)

In [31]:
best_features=pd.DataFrame(skb.scores_,columns=['Chi2 Score'])
best_features['Column']=x.columns

In [32]:
best_features.sort_values('Chi2 Score',ascending=False)

,Chi2 Score,Column
12,24.689650,Credit_History
5,7.103093,Urban
8,1.996446,1
3,1.782425,Married
0,0.988390,Education
6,0.783946,Rural
7,0.768400,0
9,0.384200,2
10,0.047199,LoanAmount
2,0.036234,Gender


#Kflod

In [34]:
from sklearn.model_selection import StratifiedKFold
skf_10=StratifiedKFold(n_splits=10,shuffle=True)

# Metrics

In [35]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [36]:
from sklearn.model_selection import cross_val_score

#Selecting Model

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [40]:
models=[]
models.append(("LogReg",LogisticRegression()))
models.append(("DesicionTree",DecisionTreeClassifier()))
models.append(("SVC",SVC()))
models.append(("kNN",KNeighborsClassifier()))
models.append(("RandomForest",RandomForestClassifier()))

In [42]:
names=[]
result=[]
scoring='accuracy'
for name,model in models:
  cv_result=cross_val_score(model,x_scaled,y,scoring=scoring,cv=skf_10)
  print("Model created: %s ------> Validation Error: %.2f" % (name,cv_result.mean()))
  print("-----------------------------------------------")

Model created: LogReg ------> Validation Error: 0.80
-----------------------------------------------
Model created: DesicionTree ------> Validation Error: 0.71
-----------------------------------------------
Model created: SVC ------> Validation Error: 0.80
-----------------------------------------------
Model created: kNN ------> Validation Error: 0.76
-----------------------------------------------
Model created: RandomForest ------> Validation Error: 0.79
-----------------------------------------------


### Testing the model with SVC , LogReg and RandomForest for Hyperparameter Tuning

# Testing various model 

## Log Reg

In [55]:
## Tuning Tol , max_iter and penalty 
log_reg=LogisticRegression(penalty='none',tol=0.0001,max_iter=200)
np.mean(cross_val_score(log_reg,x_scaled,y,cv=skf_10))

0.7979640401903755

In [58]:
## Tuning Tol , max_iter and penalty 
log_reg=LogisticRegression(penalty='l2',tol=0.00001,max_iter=500)
np.mean(cross_val_score(log_reg,x_scaled,y,cv=skf_10))

0.7961924907456371

## Random Forest

In [60]:
## Selecting few features + moderating n_estimators
features=['Education', 'ApplicantIncome', 'Self_Employed',
       'Urban', 'Rural', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']
for i in range(10,60,10):
  rfc_model=RandomForestClassifier(n_estimators=i)
  rfc_model.fit(x[features],y)
  print('Training error: %.2f -----> Validation Error: %.2f' % (rfc_model.score(x[features],y),np.mean(cross_val_score(rfc_model,x[features],y,cv=skf_10))))
  print('---------------------------------------------------')

Training error: 0.98 -----> Validation Error: 0.75
---------------------------------------------------
Training error: 1.00 -----> Validation Error: 0.78
---------------------------------------------------
Training error: 1.00 -----> Validation Error: 0.79
---------------------------------------------------
Training error: 1.00 -----> Validation Error: 0.78
---------------------------------------------------
Training error: 1.00 -----> Validation Error: 0.78
---------------------------------------------------


In [68]:
for i in range(10,60,10):
  rfc_model=RandomForestClassifier(n_estimators=i)
  rfc_model.fit(x,y)
  print('Training error: %.2f -----> Validation Error: %.2f' % (rfc_model.score(x,y),np.mean(cross_val_score(rfc_model,x,y,cv=skf_10))))
  print('---------------------------------------------------')

Training error: 1.00 -----> Validation Error: 0.75
---------------------------------------------------
Training error: 0.99 -----> Validation Error: 0.78
---------------------------------------------------
Training error: 1.00 -----> Validation Error: 0.77
---------------------------------------------------
Training error: 1.00 -----> Validation Error: 0.79
---------------------------------------------------
Training error: 1.00 -----> Validation Error: 0.78
---------------------------------------------------


## SVM 

In [73]:
from sklearn.svm import SVC
svm=SVC(C=1)
svm.fit(x_scaled,y)
print('Training error: %.2f -----> Validation Error: %.2f' % (svm.score(x_scaled,y),np.mean(cross_val_score(svm,x_scaled,y,cv=skf_10))))

Training error: 0.83 -----> Validation Error: 0.80


##SVM is our final Model

#Test Data

In [74]:
test=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Competitions/Loan Prediction/test_lAUu6dG.csv')

In [75]:
test=remove_missing_value_cat(test,si)

In [76]:
test=categorical_to_binary(test,ohe_pa,ohe_dep)

In [77]:
test=remove_missing_value_num(test,knn_impute_num)

In [78]:
test_id=test['Loan_ID']
test.drop('Loan_ID',axis=1,inplace=True)

In [79]:
len(test)

367

In [80]:
test_scaled=mms.transform(test)

In [93]:
svc_final_model=SVC(C=1)

In [94]:
svc_final_model.fit(x_scaled,y)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [95]:
pred=svc_final_model.predict(test_scaled)

In [96]:
result=pd.DataFrame()
result['Loan_ID']=test_id
result['Loan_Status']=pred

In [97]:
result['Loan_Status']=result['Loan_Status'].map({1:'Y',0:'N'})

In [99]:
result.to_csv('svm1.csv',index=False)

In [98]:
result['Loan_Status'].value_counts()

Y    304
N     63
Name: Loan_Status, dtype: int64